In [1]:
import ipywidgets as widgets
from ipyevents import Event
from IPython.display import display
import requests
import pandas as pd

STUDENT_ID = "50109581"

In [2]:
def get_dataset():
    url = "http://betelgeuse.cse.buffalo.edu/getstudentimages/50109581"
    r = requests.get(url)
    dataset = pd.DataFrame(r.json())
    return dataset

def get_options():
    url = "http://betelgeuse.cse.buffalo.edu/getdropdownoptions"
    r = requests.get(url)
    options = pd.DataFrame(r.json())
    return options

def get_examples(feature):
    url = "http://betelgeuse.cse.buffalo.edu/getexamples/"
    r = requests.get(url + feature)
    examples = pd.DataFrame(r.json())
    return examples

def get_image_widget(img_src):
    url_base = "http://betelgeuse.cse.buffalo.edu/dataset/"
    r = requests.get(url_base + img_src)
    return widgets.Image(
        value=r.content
    )

def create_feature_widget(current_feature):
    global column_widgets, feature_label, example_images
    feature_name = get_options().columns[current_feature-1]
    feature_label.value = feature_name

    column_widgets = []
    examples = get_examples(feature_name)
    for i, column in enumerate(examples.columns):
        example_widgets = [widgets.Label(f"{i+1}: " + column)]
        for index, example in examples.iterrows():
            image_widget = get_image_widget(example[column])
            example_widgets.append(image_widget)
        vbox = widgets.VBox(example_widgets)
        vbox.layout.border = '2px solid red'
        column_widgets.append(vbox)
        
    example_images.children = column_widgets

# Putting it Together

In [3]:
dataset = get_dataset()

In [19]:
current_feature = 9
current_index = 1

column_widgets = []
feature_label = widgets.Label()
feature_label.layout.display = 'flex'
feature_label.layout.justify_content = 'center'

example_images = widgets.HBox(column_widgets)
example_images.layout.display = 'flex'
example_images.layout.justify_content = 'center'

create_feature_widget(current_feature)

instance_image = get_image_widget(dataset[1][current_index-1])
instance_box = widgets.HBox([instance_image])
instance_box.layout.border = '2px solid green'
instance_box.layout.display = 'flex'
instance_box.layout.justify_content = 'center'

app = widgets.VBox([instance_box, feature_label, example_images])
app.layout.border = '2px solid blue'
app.layout.display = 'flex'
app.layout.justify_content = 'center'

#d = Event(source=app, watched_events=['click', 'keydown', 'mouseenter'])
d = Event(source=app, watched_events=['keydown'])
h = widgets.HTML(f'{current_index}/{len(dataset)} examples of {current_feature}/{len(get_options().columns)} features')

def assign(image_id, feature, value):
    url = f'http://betelgeuse.cse.buffalo.edu/updatedropdownoptions/{image_id}/{feature}/{value}'
    #h.value = url
    r = requests.get(url)
    
def handle_event(event, i=None):
    global current_index, current_feature, instance_box, h
    #h.value = str(i)
    if i is not None:
        value = str(i+1)
    else:
        value = event['key']
    
    image_id = int(dataset[0][current_index-1])
    assign(image_id, current_feature, value)
    
    current_index += 1
    if current_index > len(dataset):
        current_index = 1
        current_feature += 1
        create_feature_widget(current_feature)
        
    instance_box.children = (get_image_widget(dataset[1][current_index-1]),)
    h.value = f'{current_index}/{len(dataset)} examples of {current_feature}/{len(get_options().columns)} features'

def handle_touch_event(i):
    return lambda e: handle_event(e, i)

d.on_dom_event(handle_event)

for i, cw in enumerate(column_widgets):
    #h.value += f'{i},'
    event = Event(source=cw, watched_events=['click'])
    event.on_dom_event(handle_touch_event(i))
                            
display(app, h)

HTML(value='1/152 examples of 9/15 features')